In [78]:
import sys, re
sys.path.append('../xrun')

import fractions
from fractions import Fraction

from timeit import default_timer as timer
from pathlib import Path

import numba
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from numba import jit, cfunc
from scipy import linalg
from scipy.sparse import linalg as sparse_linalg, issparse
from sklearn.utils.sparsefuncs import mean_variance_axis
from sklearn.utils.extmath import svd_flip, safe_sparse_dot
from sklearn.metrics import pairwise_distances_argmin_min

from xrun.data.loader import load_dataset
from xrun.data.run_info import RunInfo

sns.set(style="whitegrid", font_scale=1.2)

In [2]:
def load_run_info(experiment_dir: Path):
    run_file_paths = list(experiment_dir.glob("*.json"))
    if len(run_file_paths) != 1:
        print(f"Expected a single run file in {experiment_dir} but found {len(run_file_paths)} files.")
        return None
    return RunInfo.load_json(run_file_paths[0])

def create_block_highlighter(k: int):
    def apply_highlight(col: pd.Series):
        col_index = int(col.name)
        colors = ['#b3cde0', '#2ab7ca', '#fed766'] if (col_index // k) % 2 == 0 else ['#e6e6ea', '#f6abb6', "#dec3c3"]
        return [f"background-color: {colors[(i // k) %len(colors)]}" for i in range(len(col))]
    return apply_highlight

def highlight_negative(col: pd.Series):
    colors = ['#2ab7ca', '#f6abb6']
    return [f"background-color: {colors[int('-' in val)]}" for val in col.values]

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def generate_benchmark(k: int, alpha: int, beta: float):
    """Generate benchmark dataset.

    Parameters
    ----------
    k: int
        The size of the initial block which is a square k-by-k matrix.
    alpha: int
        The number of the column blocks.
    beta: float:
        The factor for which to scale the column box.
    Returns
    -------
    np.array
        Returns a matrix of size (k^alpha x alpha*k)
    """
    # Create NxD matrix
    n = k ** alpha
    d = alpha * k
    data = np.zeros((n, d))

    # Construct the first N-by-k left-side of the matrix
    for i in range(n):
        for j in range(k):
            value = 0
            if i % k == j:
                value = (k-1) / k
            else:
                value = -1/k
            data[i,j] = value

    # Fill the rest by using the copy-stack operation
    for j in range(k, d):
        for i in range(n):
            copy_i = i // k
            copy_j = j - k
            data[i,j] = data[copy_i, copy_j]
    
    # Apply columnblock-level scaling factor beta
    if beta > 1:
        for i in range(alpha):
            start_col = i*k
            end_col = i*k + k
            beta_val = np.power(beta, float(-i))
            data[:, start_col:end_col] *= beta_val

    return data

In [3]:
result_path = Path("../data/odin-results/hardinstanceb1/sensitivity-sampling-k10-m2000/2021-09-09-14-01-39/results.txt.gz")
experiment_dir = result_path.parent

In [4]:
run_info = load_run_info(experiment_dir)

In [5]:
run_info

RunInfo(algorithm='sensitivity-sampling', dataset='hardinstanceb1', k=10, m=2000, iteration=8, randomSeed=357918331, output_dir='data/experiments/hardinstanceb1/sensitivity-sampling-k10-m2000/2021-09-09-14-01-39', command='gs/build/gs sensitivity-sampling hardinstanceb1 data/input/benchmark-k10-alpha6-beta1.00.txt.gz 10 2000 357918331 data/experiments/hardinstanceb1/sensitivity-sampling-k10-m2000/2021-09-09-14-01-39', start_time='2021-09-09T14:01:39.817304', end_time='2021-09-09T14:09:47.370186', duration_secs=487.552882, process_id=-2)

In [42]:
benchmark = load_dataset('../' + run_info.dataset_path)

Loading csv dataset from ../data/input/benchmark-k10-alpha6-beta1.00.txt.gz...
Loaded matrix of shape (1000000, 60) in 10.90 secs


In [43]:
run_info.dataset_path

'data/input/benchmark-k10-alpha6-beta1.00.txt.gz'

In [50]:
parsed_str = re.findall(r"benchmark-k(\d+)-alpha(\d+)-beta", run_info.dataset_path)[0]
k = int(parsed_str[0])
alpha = int(parsed_str[1])

k, alpha

(10, 6)

In [75]:
k = 3
alpha = 2
beta = 1
a = 0
i = 0

get_cluster_member_indices(k=k, alpha=alpha, block_index=a, cluster_label=i) + 1

array([1, 4, 7])

In [6]:
a = 0.01
k = 1

In [7]:
computed_coreset = np.loadtxt(
    fname=result_path,
    dtype=np.double,
    delimiter=" ",
    skiprows=1,
    unpack=False
)

In [8]:
coreset_weights = computed_coreset[:,0]
coreset_points = computed_coreset[:,1:]


In [9]:
coreset_points.shape

(2017, 60)

In [76]:
coreset_points.shape

(2017, 60)

In [77]:
benchmark.shape

(1000000, 60)

In [81]:
closest_points, distances = pairwise_distances_argmin_min(coreset_points, benchmark, metric="euclidean")

/home/omar/.cache/pypoetry/virtualenvs/xrun-eubxByDy-py3.8/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1450: FutureWarning: from version 1.0 (renaming of 0.25), pairwise_distances for metric='seuclidean' will require V to be specified if Y is passed.
  warnings.warn(


In [84]:
closest_points

array([538711, 728184, 128847, ..., 683606, 454614, 373564])

In [87]:
distances

array([0.        , 0.        , 0.        , ..., 7.0121979 , 6.20110735,
       5.86876959])

In [88]:
parsed_str = re.findall(r"benchmark-k(\d+)-alpha(\d+)-beta", run_info.dataset_path)[0]
k = int(parsed_str[0])
alpha = int(parsed_str[1])

k, alpha

(10, 6)

In [90]:

    
a = 0
i = 0
cluster_members = get_cluster_member_indices(k=k, alpha=alpha, block_index=a, cluster_label=i)

In [148]:
print(f"alpha: {alpha}")
for a in range(alpha):
    print(a)

alpha: 6
0
1
2
3
4
5


In [97]:
cluster_members.shape

(100000,)

In [102]:
cluster_members

array([     0,     10,     20, ..., 999970, 999980, 999990])

In [113]:
# Compute the intersection between cluster members and the coreset points: C_i^a ∩ Ω
coreset_points_in_cluster = np.intersect1d(closest_points, cluster_members)
coreset_points_in_cluster

array([   100,  16750,  17230,  18390,  29580,  34450,  43480,  45840,
        58680,  61670,  62650,  65790,  67460,  72370,  72450,  75420,
        78020,  83400,  86160,  87670,  90930,  94600, 116960, 125050,
       126600, 127570, 137500, 137850, 141180, 142510, 142890, 144360,
       148570, 148690, 161040, 162900, 169180, 173580, 173960, 177580,
       178050, 184580, 190870, 203630, 209130, 217560, 227370, 230760,
       234820, 235160, 236820, 242480, 249400, 251490, 257080, 258720,
       260180, 262000, 262780, 264040, 265050, 266590, 268990, 278330,
       290530, 292300, 301990, 314440, 327710, 346230, 346700, 348630,
       349730, 353020, 354500, 358950, 362260, 363140, 363790, 367340,
       369460, 374500, 375110, 379140, 389010, 389610, 393140, 393930,
       399100, 400040, 403850, 413090, 430020, 433660, 447670, 452050,
       453050, 465780, 466930, 472930, 473390, 480950, 481500, 482030,
       486290, 486330, 486470, 487430, 491310, 491520, 495190, 497670,
      

In [119]:
# Find the weights of coreset points in the cluster
coreset_points_in_cluster_indices = np.where(np.isin(closest_points, coreset_points_in_cluster))[0]
coreset_points_in_cluster_indices

array([  14,   27,   34,   48,   50,   80,   81,  114,  122,  127,  130,
        161,  169,  171,  173,  177,  181,  189,  190,  237,  238,  255,
        256,  266,  268,  274,  280,  288,  292,  295,  311,  315,  337,
        354,  375,  408,  424,  488,  489,  503,  505,  508,  514,  518,
        534,  552,  570,  587,  595,  596,  599,  631,  639,  645,  652,
        654,  655,  661,  663,  676,  685,  714,  725,  738,  744,  750,
        760,  777,  778,  782,  795,  800,  801,  805,  812,  825,  829,
        832,  836,  844,  852,  854,  862,  869,  878,  887,  894,  895,
        910,  921,  922,  923,  927,  930,  946,  958,  963,  967,  984,
        988,  998, 1006, 1019, 1032, 1039, 1041, 1049, 1058, 1059, 1069,
       1070, 1074, 1081, 1082, 1091, 1092, 1108, 1116, 1133, 1150, 1158,
       1176, 1180, 1182, 1186, 1187, 1189, 1190, 1215, 1227, 1229, 1242,
       1251, 1255, 1262, 1268, 1283, 1297, 1358, 1362, 1387, 1391, 1412,
       1413, 1414, 1427, 1436, 1449, 1459, 1472, 14

In [123]:
# Compute the mass of points of C_i^a in Ω
mass = np.sum(coreset_weights[coreset_points_in_cluster_indices])
mass

104107.44399999999

In [125]:
n_cluster_members = cluster_members.shape[0]
n_cluster_members

100000

In [146]:
cluster_means = []
print(f"Mass is {mass}")
for epsilon in [0.01, 0.05, 0.1, 0.3, 0.5]:
    non_deficient_threshold = (1 - epsilon) * n_cluster_members
    print(f"For epsilon {epsilon}, the threshold is {non_deficient_threshold}")
    if mass >= non_deficient_threshold:
        # Add cluster mean to the solution
        print(" - Adding cluster mean to the solution")
        cluster_mean = np.mean(benchmark[cluster_members], axis=0)
        cluster_means.append(cluster_mean)
        break

Mass is 104107.44399999999
For epsilon 0.01, the threshold is 99000.0
 - Adding cluster mean to the solution


In [145]:
np.allclose(np.array(cluster_means)[1], np.array(cluster_means)[3])

True

In [118]:
coreset_points[1898]

array([ 0.9, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1,  0.9, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
        0.9, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1,  0.9, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
        0.9, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,  0.9, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1])

In [109]:
coreset_weights[1936]

497.611

---

In [38]:
k = 4
alpha = 4
beta = 1



n = k ** alpha
a = 3

data = generate_benchmark(k=k, alpha=alpha, beta=beta)

induced_clusters = [(n // k**a) % k  for n in range(n)]


In [39]:
clusters = np.array(induced_clusters)

indices = list(np.where(clusters == 0)[0])

df_data = pd.DataFrame(data[indices])
df_data = df_data.apply(lambda r: r.apply(lambda s: str(Fraction(s).limit_denominator())))
df_data.style.apply(create_block_highlighter(k))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4
1,-1/4,3/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4
2,-1/4,-1/4,3/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4
3,-1/4,-1/4,-1/4,3/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4
4,3/4,-1/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4
5,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4
6,-1/4,-1/4,3/4,-1/4,-1/4,3/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4
7,-1/4,-1/4,-1/4,3/4,-1/4,3/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4
8,3/4,-1/4,-1/4,-1/4,-1/4,-1/4,3/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4
9,-1/4,3/4,-1/4,-1/4,-1/4,-1/4,3/4,-1/4,3/4,-1/4,-1/4,-1/4,3/4,-1/4,-1/4,-1/4
